In [19]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")


In [22]:
# Load the dataset
df = pd.read_csv("C:/Users/Sarika Kene/Desktop/Self note course/retail_store_inventory.csv")

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Create a YearMonth column for monthly grouping
df['YearMonth'] = df['Date'].dt.to_period('M').astype(str)

# Group by YearMonth and Category to get total Units Sold
monthly_sales = df.groupby(['YearMonth', 'Category'])['Units Sold'].sum().reset_index()
# Prepare to store forecasts
forecast_list = []
error_categories = []


In [23]:
# Prepare to store forecasts
forecast_list = []
error_categories = []


In [24]:
# Forecast for each category
for cat in monthly_sales['Category'].unique():
    cat_df = monthly_sales[monthly_sales['Category'] == cat]

    # Convert YearMonth to datetime index for ARIMA
    cat_df = cat_df.set_index(pd.to_datetime(cat_df['YearMonth']))

    # Only forecast if there's at least 12 months of data
    if len(cat_df) >= 12:
        try:
            model = ARIMA(cat_df['Units Sold'], order=(1, 1, 1))
            model_fit = model.fit()

            # Forecast next 6 months
            forecast = model_fit.forecast(steps=6)

            # Create forecast DataFrame
            forecast_df = pd.DataFrame({
                'YearMonth': pd.date_range(
                    start=cat_df.index[-1] + pd.DateOffset(months=1),
                    periods=6,
                    freq='M'
                ).to_period('M').astype(str),
                'Predicted Sales': forecast,
                'Category': cat
            })

            forecast_list.append(forecast_df)

        except Exception as e:
            error_categories.append((cat, str(e)))
            continue


In [28]:
# Combine all forecasted data
if forecast_list:
    forecast_all = pd.concat(forecast_list, ignore_index=True)
    forecast_all.to_excel("category_sales_forecast2.xlsx", index=False)
    print("✅ Forecast file saved: category_sales_forecast.xlsx")
else:
    print("❌ No forecasts generated.")


✅ Forecast file saved: category_sales_forecast.xlsx


In [29]:
# Print categories that failed to forecast
if error_categories:
    print("\n⚠️ Forecasting failed for the following categories:")
    for cat, err in error_categories:
        print(f"{cat}: {err}")


In [30]:
import os
print("File saved at:", os.path.abspath("category_sales_forecast2.xlsx"))

File saved at: C:\Users\Sarika Kene\category_sales_forecast2.xlsx
